In [1]:
import pandas as pd
from utilities.functions import *

# load the lookup table
geoid_lu_df = pd.read_csv('tbl/NCRPlaces_Census_04192024.csv')

# to test, limit the lookup table to a few locations to test each geography type
# we include an individual county, incorporated place, census designated place, 
# and a 1-to-many entry (Eagle River) that has multiple census tracts per single GVV ID
geoid_lu_df_test = geoid_lu_df[geoid_lu_df['name'].isin(['Eagle River', 'Fairbanks', 'Arctic Village', 'Fort Yukon'])]

In [2]:
geoid_lu_df_test

,id,name,alt_name,region,country,latitude,longitude,type,GEOIDFQ,PLACENAME,AREATYPE,COMMENT
10,AK124,Fairbanks,NaN,Alaska,US,64.8378,-147.716,community,0500000US02090,Fairbanks North Star Borough,County,Data represent information from nearest [COLUM...
45,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000201,Census Tract 2.01,Census tract,Data for this place represent multiple merged ...
46,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000202,Census Tract 2.02,Census tract,Data for this place represent multiple merged ...
47,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000204,Census Tract 2.04,Census tract,Data for this place represent multiple merged ...
48,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000205,Census Tract 2.05,Census tract,Data for this place represent multiple merged ...
49,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000206,Census Tract 2.06,Census tract,Data for this place represent multiple merged ...
77,AK22,Arctic Village,Vashrąįį K'ǫǫ,Alaska,US,68.1269,-145.538,community,1600000US0203990,Arctic Village CDP,Census designated place,Data represent information from nearest [COLUM...
166,AK130,Fort Yukon,Gwichyaa Zheh,Alaska,US,66.5647,-145.274,community,1600000US0226760,Fort Yukon city,Incorporated place,Data represent information from nearest [COLUM...


In [3]:
# use the individual fetch functions get data for GVV IDs in the test lookup table
# follow the printed URLs to QC individual values from the results table
for gvv_id in list(geoid_lu_df_test.id.unique()):
    dhc = fetch_census_data_and_compute("dhc", gvv_id, geoid_lu_df_test, print_url=True)
    print(dhc)
    acs5 = fetch_census_data_and_compute("acs5", gvv_id, geoid_lu_df_test, print_url=True)
    print(acs5)
    cdc = fetch_cdc_data_and_compute(gvv_id, geoid_lu_df_test, print_url=True)
    print(cdc)


Requesting US Census data from: https://api.census.gov/data/2020/dec/dhc?get=P12_001N,P12_002N,P12_026N,P12_003N,P12_004N,P12_005N,P12_006N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_027N,P12_028N,P12_029N,P12_030N,P12_044N,P12_045N,P12_046N,P12_047N,P12_048N,P12_049N,P9_001N,P9_002N,P9_005N,P9_006N,P9_007N,P9_008N,P9_009N,P9_010N,P9_011N&for=county:090&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
  GEOID  total_population  pct_65_plus  pct_under_18  pct_under_5  \
0   090           95655.0        11.46         23.95          7.0   

   pct_hispanic_latino  pct_white  pct_african_american  \
0                 7.65      66.05                  3.98   

   pct_amer_indian_ak_native  pct_asian  pct_hawaiian_pacislander  pct_other  \
0                       7.63       3.17                      0.61       0.85   

   pct_multi  
0      10.06  
Requesting US Census data from: https://api.census.gov/data/2021/acs/acs5/subject?get=S1810_C03_001E,S1810_C03_001M,S2701_C

In [4]:
# also test the fetch_and_merge() function, which will add the comments from the comment dict
test_results = []
test_comment_dict = create_comment_dict(geoid_lu_df_test)

for gvv_id in list(geoid_lu_df_test.id.unique()):
    test_results.append(fetch_and_merge(geoid_lu_df_test, gvv_id, test_comment_dict))
test_results_df = pd.concat(test_results)

# view just the geographic info and comments
pd.options.display.max_colwidth = 999
test_results_df[['id','name','areatype','placename','comment']]

,id,name,areatype,placename,comment
0,AK124,Fairbanks,County,Fairbanks North Star Borough,"Data represent information from nearest county (Fairbanks North Star Borough), which includes Fairbanks."
0,AK103,Eagle River,Census tract,Census Tract 2.01,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
1,AK103,Eagle River,Census tract,Census Tract 2.02,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
2,AK103,Eagle River,Census tract,Census Tract 2.04,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
3,AK103,Eagle River,Census tract,Census Tract 2.05,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
4,AK103,Eagle River,Census tract,Census Tract 2.06,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
0,AK22,Arctic Village,Census designated place,Arctic Village CDP,"Data represent information from nearest census designated place (Arctic Village CDP), which includes Arctic Village."
0,AK130,Fort Yukon,Incorporated place,Fort Yukon city,"Data represent information from nearest incorporated place (Fort Yukon city), which includes Fort Yukon."


In [5]:
# we will also test the run_fetch_and_merge() function, as this adds the additional reference data for state of AK and the US
test_run_results_df = run_fetch_and_merge(geoid_lu_df_test)
test_run_results_df

/Users/joshpaul/epa-justice/repo/epa-justice/utilities/functions.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df.index)] = ak_row
/Users/joshpaul/epa-justice/repo/epa-justice/utilities/functions.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df.index)] = us_row


,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_under_5,pct_hispanic_latino,...,pct_hd,pct_copd,pct_diabetes,pct_kd,pct_stroke,pct_no_bband,pct_no_hsdiploma,pct_below_150pov,pct_minority,comment
0,AK124,Fairbanks,County,Fairbanks North Star Borough,090,95655.0,11.46,23.95,7.00,7.65,...,4.40,5.20,7.40,2.40,2.40,11.60,5.50,14.00,31.60,"Data represent information from nearest county (Fairbanks North Star Borough), which includes Fairbanks."
0,AK103,Eagle River,Census tract,Census Tract 2.01,020000201,4318.0,7.74,26.98,7.50,9.59,...,3.00,4.30,5.20,1.80,1.70,8.90,7.10,11.50,26.90,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
1,AK103,Eagle River,Census tract,Census Tract 2.02,020000202,6384.0,12.97,25.50,6.74,7.50,...,3.70,4.70,6.00,2.10,2.10,4.70,3.90,6.90,28.20,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
2,AK103,Eagle River,Census tract,Census Tract 2.04,020000204,3582.0,12.65,21.30,4.91,5.30,...,3.30,4.20,5.60,2.00,1.80,3.30,1.90,6.90,19.70,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
3,AK103,Eagle River,Census tract,Census Tract 2.05,020000205,7421.0,7.16,30.64,8.09,9.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
4,AK103,Eagle River,Census tract,Census Tract 2.06,020000206,3413.0,13.45,28.13,6.39,6.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
0,AK22,Arctic Village,Census designated place,Arctic Village CDP,03990,151.0,7.28,31.13,3.31,0.00,...,7.10,9.40,13.30,3.50,4.30,79.70,20.20,54.70,99.10,"Data represent information from nearest census designated place (Arctic Village CDP), which includes Arctic Village."
0,AK130,Fort Yukon,Incorporated place,Fort Yukon city,26760,428.0,15.42,26.17,6.31,0.70,...,8.90,11.10,16.30,4.20,5.30,66.80,20.50,27.60,87.90,"Data represent information from nearest incorporated place (Fort Yukon city), which includes Fort Yukon."
0,AK0,Alaska,State,Alaska,02,733391.0,12.98,24.46,6.56,6.79,...,4.55,5.86,7.92,2.46,2.55,11.85,6.92,17.54,40.96,
0,US0,United States,Nation,United States,1,331449281.0,16.83,22.06,5.55,18.73,...,5.55,6.59,10.60,2.94,3.01,12.91,11.45,20.92,40.55,


In [6]:
# if test results look good, run the fetch and merge with the full lookup table
# even with multiprocessing, this might take a while to complete the many calls to the various APIs
# heavy API usage may be "throttled" by the service ... ~ 2-3 minutes seemed to be the norm here
# If you get any "no response" messages, that means you will be missing data for some places and you'll need to run this cell again
results_df = run_fetch_and_merge(geoid_lu_df)
# then aggregate any rows with duplicate IDs
aggregated_results_df = aggregate_results(results_df)

Aggregating values for AK103: Eagle River
Aggregating values for AK439: Joint Base Elmendorf-Richardson


/Users/joshpaul/epa-justice/repo/epa-justice/utilities/functions.py:200: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(results)
/Users/joshpaul/epa-justice/repo/epa-justice/utilities/functions.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df[col] = sub_df["total_population"] * sub_df[col] / 100
/Users/joshpaul/epa-justice/repo/epa-justice/utilities/functions.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [7]:
# compare the original one-to-many results with aggregated results
dups = results_df[results_df.duplicated(subset='id')]['id'].unique().tolist()
results_df[results_df['id'].isin(dups)]

,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_under_5,pct_hispanic_latino,...,pct_hd,pct_copd,pct_diabetes,pct_kd,pct_stroke,pct_no_bband,pct_no_hsdiploma,pct_below_150pov,pct_minority,comment
0,AK103,Eagle River,Census tract,Census Tract 2.01,020000201,4318.0,7.74,26.98,7.50,9.59,...,3.0,4.3,5.2,1.8,1.7,8.9,7.1,11.5,26.9,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
1,AK103,Eagle River,Census tract,Census Tract 2.02,020000202,6384.0,12.97,25.50,6.74,7.50,...,3.7,4.7,6.0,2.1,2.1,4.7,3.9,6.9,28.2,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
2,AK103,Eagle River,Census tract,Census Tract 2.04,020000204,3582.0,12.65,21.30,4.91,5.30,...,3.3,4.2,5.6,2.0,1.8,3.3,1.9,6.9,19.7,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
3,AK103,Eagle River,Census tract,Census Tract 2.05,020000205,7421.0,7.16,30.64,8.09,9.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
4,AK103,Eagle River,Census tract,Census Tract 2.06,020000206,3413.0,13.45,28.13,6.39,6.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
0,AK439,Joint Base Elmendorf-Richardson,Census tract,Census Tract 9801,020980100,4592.0,0.68,25.07,11.78,16.86,...,NaN,NaN,NaN,NaN,NaN,0.7,1.6,7.9,45.9,Data for this place represent multiple merged census tracts: Census Tract 9801 and Census Tract 9802
1,AK439,Joint Base Elmendorf-Richardson,Census tract,Census Tract 9802,020980200,6725.0,2.13,25.25,10.51,17.67,...,NaN,NaN,NaN,NaN,NaN,7.1,2.5,19.6,44.8,Data for this place represent multiple merged census tracts: Census Tract 9801 and Census Tract 9802


In [8]:
aggregated_results_df[aggregated_results_df['id'].isin(dups)]

,id,name,areatype,placename,GEOID,total_population,pct_65_plus,pct_under_18,pct_under_5,pct_hispanic_latino,...,pct_hd,pct_copd,pct_diabetes,pct_kd,pct_stroke,pct_no_bband,pct_no_hsdiploma,pct_below_150pov,pct_minority,comment
423,AK103,Eagle River,Census tract,"Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, Census Tract 2.06","020000201, 020000202, 020000204, 020000205, 020000206",25118.0,10.37,27.03,6.96,7.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Data for this place represent multiple merged census tracts: Census Tract 2.01, Census Tract 2.02, Census Tract 2.04, Census Tract 2.05, and Census Tract 2.06"
424,AK439,Joint Base Elmendorf-Richardson,Census tract,"Census Tract 9801, Census Tract 9802","020980100, 020980200",11317.0,1.54,25.18,11.03,17.34,...,NaN,NaN,NaN,NaN,NaN,4.5,2.13,14.85,45.25,Data for this place represent multiple merged census tracts: Census Tract 9801 and Census Tract 9802


In [9]:
# save to CSV
aggregated_results_df.to_csv('tbl/data_to_export.csv', index=False)